# Process Practice

**Uniprogramming** (old): Only one process running at a time

**Multiprogramming**: Two processes can run in parallel or concurrently

--> Modern OS try to run everything in parallel and concurrently

### The <mark>**Scheduler**</mark> Decides When to Switch to a process and run it (on a specific CPU core)

To create a process, the OS has to at least load it into memory

When it's waiting, the scheduler (coming later) decides when it’s running

### The Core Secheduling Loop (Switching)

1. Pause the currently running process
2. Save its state, so you can restore it later
3. Get the next process to run from the scheduler
4. Load the next process’ state and let that run

There are 2 ways to do this:

- **Cooperative multitasking**: Let Processes control Themselves (The processes use a system call to tell the OS to pause it)
- <mark>**True multitaksing**</mark>: The Operating System Pause (The OS retains control and pauses processes)

Modern OS uses True multitasking. The OS can:
- Give processes set time slices
- Wake up periodically using interrupts to do scheduling

Swapping process --> <mark>**Context Switching**</mark>

- Save all the values, registers, or using stack (push, pop, etc.) using the current CPU

- Note: Context Switching purely means overhead (not our goal)  
--> There should be checks if the program ever uses float, etc.

- There is hardware support to save as little as possible

***
## A New API - `pipe`

    int pipe (int pipefd[2]);

Returns 0 on success, -1 on failure 

Takes an int array of 2 int (will set these if success)

--> Forms a one way comminication channel using 2 fds (that it newly opens)

- pipefd[0] is the read end of the pipe
- pipefd[1] is the write end of the pipe

#### Aside: Using & in Your Shell
- If you use & at the end of your command, your shell will start that process and return: e.g. `sleep 1 &`
- It outputs the `pid` and lets you know when it’s finished
- The `|` character creates a pipe between two processes

### EXAMPLE: Pipe

In [3]:
%%file run.c

#include <errno.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>

// This function is just to handle the error checking after any system call
void check(int ret, const char* message) {
    if (ret != -1) {
        return;
    }
    int error = errno;      // More info: I want to exit and set my status to errno that causes the error
    perror(message);        // Because perror is a system call, can actually set errno as well, so we have to save errno to error
    exit(error);
}

int main(void) {
    int fds[2];
    check(pipe(fds), "pipe");
    pid_t pid = fork();
    check(pid, "fork");

    // Fork will create 2 file descriptors for each process
    // 3 = fds[0] = the read end of the pipe
    // 4 = fds[1] = the write end of the pipe
    
    // Parent
    if (pid > 0) {
        const char* str = "Howdy child";
        int len = strlen(str);
        int bytes_written = write(fds[1], str, len);
        check(bytes_written, "write");
    }
    // Child
    else {
        char buffer[4096];
        int bytes_read = read(fds[0], buffer, sizeof(buffer));
        check(bytes_read, "read");
        printf("Child read: %.*s\n", bytes_read, buffer);
    }

    close(fds[0]);      
    close(fds[1]);

    return 0;
}

Overwriting run.c


- It doesn't matter which process runs first.  
`read` is blocking, meaning that even if `read` is run first from the child, and there's nothing to read yet, it will go to sleep. Then, after the parent `write`, the child process can still `read` it later

- `pipe` is not a file, it's like writting to the buffer in memory by the kernel. However, the data can still be referred to using file descriptors

- after `pipe()` and then `fork()`, both processes have fds = 3 and fd = 4 opened --> Still have to `close` for both processes  
HOWEVER, we can think of file descriptors like pointers --> fd = 3 of parent and child both refer to the same thing (but yes, we still need to `close` on both processes)

<mark>**IMPORTANT**</mark>: IF we do not `write` in parent: the parent will return, we will have an orphan that always keep waiting for the `read`  
This will NOT TIMEOUT, because the child process still has fds[1] open (the write end) --> it will still wait and it doesn't get cleaned automatically by the kernel

Therefore, a better practice is we modify the code to:



In [2]:
%%file run.c

if (pid > 0) {
    close(fds[0]);
    //...
    int bytes_written = write(fds[1], str, len);
    //...
    close(fds[1]);
}
else {
    close(fds[1]);
    //...
    int bytes_read = read(fds[0], buffer, sizeof(buffer));
    //...
    close(fds[0]);
}

Overwriting run.c


This is because I closed fds[1] on the child side, so the `read` will not just stay there forever if something happens to the parent not writing to the pipe

***
### FINAL 2022 Q1

For each program shown below, state whether it will produce the same output each time it is run, or whether it may produce different outputs when run multiple times.  
Explain why the program behaves like this

In [4]:
%%file run.c

int main() {
    int i = 4;
    while (i != 0) {
        int pid = fork();
        if (pid == 0) {
            i--;
        }
        else {
            printf("%d\n", i);
            exit(0);
        }
    }
    return 0;
}

Overwriting run.c



| process 100 | process 101 | process 102  | process 103   |
| :---        | :---       | :---          | :---          | 
| i = 4      | i = 3       | i = 2   | i = 1 |
| pid = 101   | pid = 102  | pid = 103 | pid = 0 | 
| exited   |  exited | exited | exited |

Print result:

4
2
3
1

--> It is not guaranteed that the parent will always run before a child --> Different results each time we run

### FINAL 2022 Q1

What happens now?

In [6]:
%%file run.c

int main() {
    int i = 4;
    while (i != 0) {
        int pid = fork();
        if (pid == 0) {
            i--;
        }
        else {
            waitpid(pid, NULL, 0);
            printf("%d\n", i);
            exit(0);
        }
    }
    return 0;
}

Overwriting run.c


Now, the parent process will always wait for the child process to die before printing anything
--> Result will be 1 2 3 4